In [1]:
from model.EPInformer import EPInformer_v2, enhancer_predicor_256bp

In [30]:
import pandas as pd
from tqdm import tqdm

### Download ABC element-gene data for K562

In [ ]:
!wget https://www.encodeproject.org/files/ENCFF635RHY/@@download/ENCFF635RHY.bed.gz -O ./data/K562_enhancer_gene_links.bed.gz

In [ ]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz  -P ./data/

In [8]:
enhancer_gene_k562 = pd.read_csv('./data/K562_enhancer_gene_links.bed.gz', sep='\t')
enhancer_gene_k562_100kb = enhancer_gene_k562[(enhancer_gene_k562['distance']<=100_000)&(enhancer_gene_k562['distance']>1000)].reset_index()

In [10]:
enhancer_gene_k562_100kb.to_csv('./data/K562_enhancer_gene_links_100kb.tsv', index=False, sep='\t')

In [4]:
enhancer_gene_k562_100kb = pd.read_csv('./data/K562_enhancer_gene_links_100kb.tsv', sep='\t')

In [25]:
enhancer_gene_k562_100kb.columns

Index(['index', '#chr', 'start', 'end', 'name', 'class', 'activity_base',
       'normalized_h3K27ac', 'normalized_dhs', 'activity_base_squared',
       'TargetGene', 'TargetGeneTSS', 'TargetGeneExpression',
       'TargetGenePromoterActivityQuantile', 'TargetGeneIsExpressed',
       'normalized_dhs_b', 'normalized_h3k27ac', 'distance', 'isSelfPromoter',
       'powerlaw_contact', 'powerlaw_contact_reference', 'hic_contact',
       'juicebox_contact_values', 'hic_contact_pl_scaled', 'hic_pseudocount',
       'hic_contact_pl_scaled_adj', 'ABC.Score.Numerator', 'ABC.Score',
       'powerlaw.Score.Numerator', 'powerlaw.Score', 'CellType',
       'hic_contact_squared', 'Ensembl_ID'],
      dtype='object')

In [5]:
GATA1_PE = enhancer_gene_k562_100kb[enhancer_gene_k562_100kb['TargetGene'] == 'GATA1']

In [8]:
from kipoiseq import Interval
import pyfaidx
import kipoiseq
import numpy as np
import pandas as pd
import pyranges as pr
# from Bio.Seq import Seq

def df_to_pyranges(df, start_col='start', end_col='end', chr_col='chr', start_slop=0, end_slop=0):
    df['Chromosome'] = df[chr_col]
    df['Start'] = df[start_col] - start_slop
    df['End'] = df[end_col] + end_slop
    return(pr.PyRanges(df))

class FastaStringExtractor:
    def __init__(self, fasta_file):
        self.fasta = pyfaidx.Fasta(fasta_file)
        self._chromosome_sizes = {k: len(v) for k, v in self.fasta.items()}

    def extract(self, interval: Interval, **kwargs) -> str:
        # Truncate interval if it extends beyond the chromosome lengths.
        chromosome_length = self._chromosome_sizes[interval.chrom]
        trimmed_interval = Interval(interval.chrom,
                                    max(interval.start, 0),
                                    min(interval.end, chromosome_length),
                                    )
        # pyfaidx wants a 1-based interval
        sequence = str(self.fasta.get_seq(trimmed_interval.chrom,
                                          trimmed_interval.start + 1,
                                          trimmed_interval.stop).seq).upper()
        # Fill truncated values with N's.
        pad_upstream = 'N' * max(-interval.start, 0)
        pad_downstream = 'N' * max(interval.end - chromosome_length, 0)
        return pad_upstream + sequence + pad_downstream

    def close(self):
        return self.fasta.close()
    
def one_hot_encode(sequence):
    return kipoiseq.transforms.functional.one_hot_dna(sequence).astype(np.float32)

In [23]:
def encode_promoter_enhancer_links(gene_enhancer_df, fasta_path = './data/hg38.fa', max_n_enhancer = 60, max_distanceToTSS = 100_000, max_seq_len=2000, add_flanking=False):
    fasta_extractor = FastaStringExtractor(fasta_path)
    gene_pe = gene_enhancer_df.sort_values(by='distance')
    row_0 = gene_pe.iloc[0]
    gene_name = row_0['TargetGene']
    gene_tss = row_0['TargetGeneTSS']
    chrom = row_0['#chr']
    target_interval = kipoiseq.Interval(chrom, int(gene_tss-max_seq_len/2), int(gene_tss+max_seq_len/2))

    promoter_seq = fasta_extractor.extract(target_interval)
    promoter_code = one_hot_encode(promoter_seq)
    enhancers_code = np.zeros((max_n_enhancer, max_seq_len, 4))
    enhancer_activity = np.zeros(max_n_enhancer)
    enhancer_distance = np.zeros(max_n_enhancer)
    enhancer_contact = np.zeros(max_n_enhancer)
    # set distance threshold
    gene_pe = gene_pe[(gene_pe['distance'] > max_seq_len/2)&(gene_pe['distance'] <= max_distanceToTSS)]
    e_i = 0
    gene_element_pair = []
    for idx, row in gene_pe.iterrows():
        if pd.isna(row['start']):
            continue
        if e_i >= max_n_enhancer:
            break
        enhancer_start = int(row['start'])
        enhancer_end = int(row['end'])
        enhancer_center = int((row['start'] + row['end'])/2)
        enhancer_len = enhancer_end - enhancer_start
        # put sequence at the center
        if add_flanking:
            enhancer_target_interval = kipoiseq.Interval(chrom, enhancer_center-int(max_seq_len/2), enhancer_center+int(max_seq_len/2))
            enhancers_code[e_i][:] = one_hot_encode(fasta_extractor.extract(enhancer_target_interval))
        else:
            # enhancers_signal = np.zeros((max_n_enhancer, max_seq_len))
            if enhancer_len > max_seq_len:
                enhancer_target_interval = kipoiseq.Interval(chrom, enhancer_center-int(max_seq_len/2), enhancer_center+int(max_seq_len/2))
                enhancers_code[e_i][:] = one_hot_encode(fasta_extractor.extract(enhancer_target_interval))
            else:
                code_start = int(max_seq_len/2)-int(enhancer_len/2)
                enhancer_target_interval = kipoiseq.Interval(chrom, enhancer_start, enhancer_end)
                enhancers_code[e_i][code_start:code_start+enhancer_len] = one_hot_encode(fasta_extractor.extract(enhancer_target_interval))
        # put sequence from the start
        enhancer_activity[e_i] = row['activity_base']
        enhancer_distance[e_i] = row['distance']
        enhancer_contact[e_i] = row['hic_contact']*100
        gene_element_pair.append([gene_name, row['name']])
        e_i += 1
    # print(promoter_signals.shape, enhancers_signal.shape)
    pe_code = np.concatenate([promoter_code[np.newaxis,:], enhancers_code], axis=0)
    gene_element_pair = pd.DataFrame(gene_element_pair, columns=['gene', 'element'])
    return pe_code, enhancer_activity, enhancer_distance, enhancer_contact, gene_name, gene_element_pair

In [24]:
pe_code, enhancer_activity, enhancer_distance, enhancer_contact, gene_name, pair_df =  encode_promoter_enhancer_links(GATA1_PE)

In [57]:
# enhancer_gene_k562_100kb[enhancer_gene_k562_100kb['#chr'] == 'chrX']['TargetGene'].unique()
mRNA_feauture = pd.read_csv('./data/mRNA_halflife_features.csv', index_col='Gene name')
promoter_signals = pd.read_csv('./data/GeneList_K562.txt', index_col='symbol', sep='\t')
promoter_signals['PromoterActivity'] = np.sqrt(promoter_signals['H3K27ac.RPM.TSS1Kb']*promoter_signals['DHS.RPM.TSS1Kb'])

In [58]:
mRNA_feauture.columns

Index(['chrom', 'TSS', 'gene_id', 'Gene description', 'UTR5LEN_log10zscore',
       'CDSLEN_log10zscore', 'INTRONLEN_log10zscore', 'UTR3LEN_log10zscore',
       'UTR5GC', 'CDSGC', 'UTR3GC', 'ORFEXONDENSITY'],
      dtype='object')

In [59]:
gene_list = ['GATA1', 'WWC3', 'PRDX4', 'FAM50A']

In [66]:
ensid_pair_list = []
mRNA_feats = ['UTR5LEN_log10zscore',
       'CDSLEN_log10zscore', 'INTRONLEN_log10zscore', 'UTR3LEN_log10zscore',
       'UTR5GC', 'CDSGC', 'UTR3GC', 'ORFEXONDENSITY']
for gene in tqdm(gene_list):
    gene_df = enhancer_gene_k562_100kb[enhancer_gene_k562_100kb['TargetGene'] == gene]
    seq_code, act_list, dist_list, contact_list, gene_name, ensid_element_pair = encode_promoter_enhancer_links(gene_df, max_seq_len=2000, max_n_enhancer=60, max_distanceToTSS=150_000, add_flanking=False)
    # print(seq_code.shape, act_list.shape, dist_list.shape, contact_list.shape)
    mRNA_promoter_feat = np.array(list(mRNA_feauture.loc[gene, mRNA_feats].values) + [promoter_signals.loc[gene, 'PromoterActivity']])
    # data = np.array([seq_code, act_list, dist_list, contact_list])
    ensid_pair_list.append(ensid_element_pair)

100%|██████████| 4/4 [00:00<00:00, 17.57it/s]

(61, 2000, 4) (60,) (60,) (60,)
(61, 2000, 4) (60,) (60,) (60,)
(61, 2000, 4) (60,) (60,) (60,)
(61, 2000, 4) (60,) (60,) (60,)


[-0.1119840902305165,
 -0.0962541765981016,
 -0.3167160428224916,
 -0.6630045833064119,
 0.649,
 0.613,
 0.542,
 4.05,
 13.215448065626719]